# Neural Networks Experiments

## Students: <br>
Tomás Rojas <br>
Matías Montagna <br>
Alonso Utreras

## The objective of this notebook is to show the results from different models, including different inputs, but trying to get the same output.

In [ ]:
import pandas as pd
import torch
import torch.optim as optim
from torch import nn
import numpy as np
from sklearn.model_selection import train_test_split
import log_info as log


# Get data
all_data = pd.read_csv(".\data_product_temp_mp25")

all_data = all_data.dropna()
X = all_data[all_data.columns[-4]]
Y = all_data[all_data.columns[-2]]
# Y = all_data[all_data.columns[:-4]].join(Y) # para que tenga todo

### Dividing data into train, test and validation sets
We chose a train size of 70% of all data, while 15% corresponds to test and 15% to validation data.

In [ ]:
test_data_size = int(len(all_data) * 0.7)

train_data = all_data[:-test_data_size]
test_data = all_data[-test_data_size:]

X_train, X_2, y_train, y_2 = train_test_split(X, Y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_2, y_2, test_size=0.3, random_state=42)



### Setting device to work with. Use cuda if available.

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

x_train_np = np.asarray(X_train.values)
y_train_np = np.asarray(y_train.values)

X_train = torch.from_numpy(x_train_np)
y_train = torch.from_numpy(y_train_np)

### Normalizing train data:
We read here https://stackabuse.com/time-series-prediction-using-lstm-with-pytorch-in-python/
that it is important to normalize data when working with time series. 



In [ ]:
from sklearn.preprocessing import MinMaxScaler


# # scaler = MinMaxScaler(feature_range=(-1, 1))
# # train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1))

# # # Transforming data into tensors
# # train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)


## Classes for training our models:

## First model, which is a simple ff NN using just date.
Simple feedforward NN
Input: Date

In [ ]:
import torch.nn as nn


# a = m.SimpleFCModel(1, 1)
class SimpleDoubleModel(nn.Module):
    """ Simple feedforward network of two fully connected layers"""
    def __init__(self, input_size, hidden_size, output_size=1, model_name='SimpleDoubleModel'):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.log_sigmoid1 = torch.nn.LogSigmoid()
        self.fc2 = nn.Linear(in_features=hidden_size, out_features=output_size)
        self.name = model_name

    def forward(self, input):
        hidden_preds = log_sigmoid1(self.fc1(input))
        predictions = self.fc2(hidden_preds)

        return predictions

# Setting model to use and its name
modelo_1_temp_pm = SimpleDoubleModel(1, 8, 1, "Temp PM Sequential 2 layers")
model_name = "Temp PM Sequential 2 layers"
loss_1 = torch.nn.MSELoss()

# Setting model, loss and optimizer
model = modelo_1_temp_pm
loss = loss_1.to(device)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters())

# Setting logger
Logger = log.LogInfo(
    model=model, 
    model_name="temp_PM25_Sequential_double_layer"
    )

# Hyperparamters
n_epochs = 5

## Training function

In [ ]:
import time

def init_weights(model):
    # Inicializamos los pesos como aleatorios
    for name, param in model.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1) 

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# TODO: Completar
def batcher(training_data, batch_size=365*24):
    """TODO: FIX for our data. We still don't know  how our data is going to be.  """
    inout_seq = []
    L = len(training_data)
    for i in range(L-batch_size):
        train_seq = input_data[i:i+batch_size]
        train_label = input_data[i+batch_size:i+batch_size+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq


def train(model, x_train, y_train, optimizer, loss_function, epochs=5, batch_size=365*24):
    model.train()
    total_loss = 0

    for i in range(epochs):
        epoch_loss = 0
        best_test_loss = float('inf')

        for x_i, y_i in zip(x_train, y_train):
            optimizer.zero_grad()
            y_pred = model(x_i)

            loss = loss_function(y_pred, y_train)
            epoch_loss += loss.item()
            loss.backward()
            optimizer.step()

        # Save results from the best trained model
        if epoch_loss < best_test_loss:
            best_test_loss = epoch_loss
            torch.save(model.state_dict(), '{}.pt'.format(model.name))
            
        total_loss += epoch_loss
        print(f'epoch: {i} loss: {epoch_loss:10.8f}')

    print(f'Average loss: {total_loss/len(x_train):4f}')
    return total_loss

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def test(model, x_test, y_test, loss_function, batch_size=365*24):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for x_i, y_i in zip(x_test, y_test):
            # predict data using the given model
            prediction = model(x_i)
            # Compute loss
            total_loss += loss_function(prediction, y_i).item()

    print(total_loss)

    return total_loss

def load_best_model(model):
    return model.load_state_dict(torch.load(f'{model.name}.pt'))

### Execute Training and Testing:

In [ ]:
# Execute training
def execute_training(model, x_train, y_train, x_test, y_test, optimizer, loss_function, logger, n_epochs=5, batch_size=365*24):
    # Train

    start_time = time.time()
    train_loss = train(model, x_train, y_train, optimizer, loss_function, n_epochs, batch_size)
    end_time = time.time();
    train_time = end_time - start_time

    print(f'Training time = {train_time}')
    print(f'Train Loss: {train_loss}')


    # # Test
    start_time = time.time()

    test_loss = test(model, x_test, y_test, loss_function, batch_size)
    end_time = time.time()
    test_time = end_time - start_time 

    print(f'\t Val. Loss: {test_loss:.3f}')

    logger.model_loss['train'] = train_loss
    logger.model_loss['test'] = test_loss
    logger.set_train_test_size(x_train, x_test)
    logger.set_training_time(train_time)


In [ ]:
# Clean CUDA RAM
torch.cuda.empty_cache()

## TODO: Plot predictions v/s real data

In [ ]:
# X_train = X_train.transpose(0, 1)
# y_train = y_train.transpose(0, 1)
X_tensor = X_train.view(-1, 1)
y_tensor = y_train.view(-1, 1)
# X_tensor = X_tensor.transpose(0, 1)
# y_tensor = y_tensor.transpose(0, 1)

In [ ]:
X_train = X_tensor[:3000].to(device)
y_train = y_tensor[:3000].to(device)
X_test = X_tensor[3000:4000].to(device)
y_test = y_tensor[3000:4000].to(device)

In [ ]:
print(f'The current model contains {count_parameters(model)} trainable parameters.')     
model = model.double()
model.apply(init_weights)

execute_training(
    model=model,
    x_train=X_train,
    y_test=y_train,
    x_test=X_test,
    y_test=y_test,
    optimizer=optimizer,
    loss_function=loss,
    logger=Logger,
    n_epochs=n_epochs
    )

In [ ]:
load_best_model(model)

### Plotting predicted data vs real data. Also logging results into a logfile

In [ ]:
import matplotlib.pyplot as plt
import logging

logging.basicConfig(filename=f'./data/{model_name}.log', level=logging.ERROR)

# Transform tensors to numpy arrays
y_pred = model(X_test)
X_test_np = X_test.cpu().detach().numpy()
y_test_np = y_test.cpu().detach().numpy()
y_pred_np = y_pred.cpu().detach().numpy()


fig, ax = plt.subplots()

# plot both real values and prediction values
results_plot = plt.scatter(X_test_np, y_test_np, marker="*", label="results")
pred_plot = plt.scatter(X_test_np, y_pred_np, marker="+", label="predictions")

ax.legend()            
plt.xlabel("temperatura")
plt.ylabel("mp2.5")

# plot results and store it in results directory
plt.show()
plt.savefig(f'./results/{model.name}')